In [13]:
import requests
import json
import re
from datetime import datetime as dt
from dateutil.parser import parse
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [14]:
username = 'XXX'
password = 'XXX'
Server = "http://wsc.aquaticinformatics.net/AQUARIUS/publish/v2/"
server_pro = "https://wsc.aquaticinformatics.net/AQUARIUS/Provisioning/v1/"
save_to = '/Users/shg096/Desktop/Rating_Curve.csv'
# list of stations,
station_list = ['05BB001','05HG001','05CK004']; # can be loaded from a file

In [15]:
def generateRatingCurve(stationIDs, token):
    # for each station
    RC_df = pd.DataFrame() # initializing the date frame that holds the information of rating curves
    RC_df_temp = pd.DataFrame() # initializing the temporary data frame
    # populating the temporal data frame
    RC_df_temp = RC_df_temp.append({"Rating_Id": 1,\
                                    "Type":1,\
                                    "Remarks":1,\
                                    "InputValue":1,\
                                    "OutputValue":1,\
                                    "StartTime":1,\
                                    "EndTime":1,\
                                    "Station_Id":1,\
                                    "locationName":1},ignore_index=True)
    
    for station in stationIDs:
        # load the rating curve infromation
        print (station)
        
        try:
            req = requests.get(Server + 'GetRatingCurveList?RatingModelIdentifier='+\
                               'Stage-Discharge.Rating Curve@' + station + '&token=' + token)
            RatingCurves = req.json()['RatingCurves']
        except:
            print ("Failed to load Field Visit Data for Station:" + station)
            continue
            
        try:
            req = requests.get(Server + 'GetLocationData?LocationIdentifier=' + station + '&token=' + token)
            locationName = req.json()['LocationName']
        except:
            print ("Failed to get station name for station:" + station)
            continue

        ## reading the data
        for i in range(len(RatingCurves)): # loop over number of rating curve available
            # get the entire information of a Rating Curve Id
            RatingCurve = RatingCurves[i] # get the rating curve information
            # get the general information of the the rating curve
            RatingCurveId = RatingCurve['Id'] # save the rating curve number
            RatingCurveType = RatingCurve['Type'] # type of the rating curve, Linear, Logarithmic
            RatingCurveRemarks = RatingCurve['Remarks'] # if any remarks exists
            RatingCurvePeriodsOfApplicability = RatingCurve ['PeriodsOfApplicability'][0]
            RatingCurvePeriodsOfApplicabilityStartTime = RatingCurvePeriodsOfApplicability['StartTime']
            RatingCurvePeriodsOfApplicabilityEndTime = RatingCurvePeriodsOfApplicability['EndTime']
            RatingCurveBaseRatingTable = RatingCurve['BaseRatingTable'] # get the Base Rating Table
            
            for j in range(len(RatingCurveBaseRatingTable)): # loop over the numner of point in the rating curve
                point = RatingCurveBaseRatingTable[j] # get the information of the point
                # populating the data frame
                RC_df_temp ['Rating_Id'] = RatingCurveId # rating curve Id
                RC_df_temp ['Type'] = RatingCurveType # Rating Curve type, Linear or logarithmic
                RC_df_temp ['Remarks'] = RatingCurveRemarks # Remarks
                RC_df_temp ['InputValue'] = point['InputValue'] # elevation
                RC_df_temp ['OutputValue'] = point['OutputValue'] # discharge
                RC_df_temp ['StartTime'] = RatingCurvePeriodsOfApplicabilityStartTime #start of rating curve
                RC_df_temp ['EndTime'] = RatingCurvePeriodsOfApplicabilityEndTime # end of rating curve
                RC_df_temp ['Station_Id'] = station # name of the station
                RC_df_temp ['locationName'] = locationName # name of the station
                # appending to the general dataframe
                RC_df = RC_df.append(RC_df_temp) # populate the infromation of the rating curve point
                
    print(RC_df) # it can be saved
    return RC_df
        
def main():
    stationIDs = station_list
    token = getToken(username, password)
    RC_df = generateRatingCurve(stationIDs, token)
    RC_df.to_csv(save_to)
    
def getToken(userName, password):
    s = requests.Session()
    data = '{"Username": "' + userName + '", "EncryptedPassword": "' + password + '", "Locale": ""}'
    url = server_pro + 'session'
    s.get(url)
    headers = {'Content-Type': 'application/json', 'Accept': 'application/json'}
    r = s.post(url, data=data, headers=headers)
    token = r.text
    return token

if __name__ == "__main__":
    main()

05BB001
05HG001
05CK004
                              EndTime  InputValue  OutputValue   Rating_Id  \
0   1999-01-01T00:00:00.0000000-07:00     1.45000         4.10      14.000   
0   1999-01-01T00:00:00.0000000-07:00     1.53000         4.90      14.000   
0   1999-01-01T00:00:00.0000000-07:00     1.63000         6.40      14.000   
0   1999-01-01T00:00:00.0000000-07:00     1.76000         9.00      14.000   
0   1999-01-01T00:00:00.0000000-07:00     1.87000        12.30      14.000   
..                                ...         ...          ...         ...   
0   2008-07-08T11:05:00.0000000-07:00     5.45254      1750.00  17.000-SC7   
0   9999-12-31T23:59:59.9999999+00:00     1.04200         2.79       18.00   
0   9999-12-31T23:59:59.9999999+00:00     1.59400        52.73       18.00   
0   9999-12-31T23:59:59.9999999+00:00     2.44300       190.00       18.00   
0   9999-12-31T23:59:59.9999999+00:00     4.12100      1773.00       18.00   

                                       